In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!ngrok config add-authtoken 2pq42yCCr5Oj3j7D2H08bdWWCOh_6W1QQnvpeMg365LtWXdNU
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
!pip install fastapi uvicorn python-multipart pyngrok jinja2
!pip install nest_asyncio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-o9q04hm7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-o9q04hm7
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 61.1 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.7 MB/s eta 0:00:00


In [2]:
#Load the Model-Qwen 7B
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
     "Qwen/Qwen2-VL-7B-Instruct",
     torch_dtype=torch.bfloat16,
     #attn_implementation="flash_attention_2",
     device_map="auto",
 )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [3]:
"""
This FastAPI web application processes product and freshness details extracted from images. It includes:

1. SQLite database management to store product details (name, MRP, expiry date, etc.) and freshness information (produce type, freshness index, and expected lifespan).
2. API endpoints to fetch product and freshness data from the databases.
3. Functions to extract and save product information and freshness details from images using AI models.
4. Error handling to ensure missing details are marked as "NA".
5. User-friendly interface for easy image uploads and viewing product data.
6. Clear sections to display product details (e.g., name, MRP, expiry date) and freshness information.
7. Option to download or save processed data for inventory management.
"""

import os
import base64
import io
import sqlite3
import pytz
from fastapi import FastAPI, File, UploadFile, Request, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse, StreamingResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from transformers import AutoProcessor, AutoModelForSeq2SeqLM
import pandas as pd
import uvicorn
from PIL import Image
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import json
from datetime import datetime
from typing import List



app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

templates = Jinja2Templates(directory="templates")

IMAGE_FOLDER = os.path.join(os.getcwd(), 'images')
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)


# SQLite database setup for product details
def connect_to_db(db_name="product_details.db"):
    return sqlite3.connect(db_name)

# SQLite database setup for freshness details
def connect_to_freshness_db(db_name="freshness_details.db"):
    return sqlite3.connect(db_name)

# Creating product_details table
def create_table(conn):
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS product_details (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        product_name TEXT,
                        mrp REAL,
                        net_content TEXT,
                        expiry_date TEXT,
                        quantity INTEGER,
                        timestamp TEXT,
                        expired TEXT,
                        expected_life TEXT)''')
    conn.commit()

# Creating freshness_details table
def create_freshness_table(conn):
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS freshness_details (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        Timestamp TEXT,
                        Produce TEXT,
                        Freshness INTEGER,
                        Expected_Life_Span INTEGER)''')
    conn.commit()

# Saving extracted product details in the Database
def save_multiple_to_database(conn, products):
    cursor = conn.cursor()
    for product in products:
        cursor.execute('''INSERT INTO product_details (
                            product_name, mrp, net_content, expiry_date, quantity, timestamp, expired, expected_life)
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                       (product["Product Name"], product["MRP"], product["Net Content"],
                        product["Expiry Date"], product["Quantity"], product["Timestamp"],
                        product["Expired"], product["ExpectedLife"]))
    conn.commit()

# Saving freshness details in the Database
def save_multiple_to_freshness_database(conn, produce_details):
    cursor = conn.cursor()
    for produce in produce_details:
        cursor.execute('''INSERT INTO freshness_details (
                           Timestamp, Produce, Freshness, Expected_Life_Span)
                          VALUES (?, ?, ?, ?)''',
                       (produce["Timestamp"], produce["Produce"], produce["Freshness"],
                        produce["Expected_Life_Span"]))
    conn.commit()



# Function to extract the details
def extract_product_details_from_image(image_path):
    message = {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": """
Please extract the following details for each product in the image:
1. Product Name (if missing, return "NA")
2. Maximum Retail Price (MRP, (if missing, return "NA"))
3. Expiry Date (DD/MM/YY or MM/YY format, if missing, return "NA"))
4. Net Content (if missing, return "NA")

If any detail is missing, return "NA".

Additionally, for each distinct product, return the quantity (number of occurrences) of that product in the image.
{
    "products": [
        {"Product Name": "<value>", "MRP": "<value(only real numbers)>", "Expiry Date": "<value>", "Net Content": "<value>", "Quantity": <count>}
        ...
    ]
}"""}
        ],
    }


    text = processor.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info([message])
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")


    generated_ids = model.generate(**inputs, max_new_tokens=300)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )


    if output_text:


        output = output_text[0].strip()
        cleaned_string = output.strip()
        cleaned_string = cleaned_string.replace('```json', '').replace('```', '').strip()
        cleaned_string = cleaned_string.strip()

        try:

            result = json.loads(cleaned_string)


        except json.JSONDecodeError:


            result = {
                "products": [{
                    "Product Name": "NA",
                    "MRP": "NA",
                    "Expiry Date": "NA",
                    "Net Content": "NA",
                    "Quantity": 0
                }]
            }
        return result


    return {
        "products": [{
            "Product Name": "NA",
            "MRP": "NA",
            "Expiry Date": "NA",
            "Net Content": "NA",
            "Quantity": 0
        }]
    }


# Function to extract the freshness details
def extract_freshness_details_for_multiple(image_path):

    message = {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": """
Analyze the provided image and identify all distinct fresh produce items present in the image. For each item, provide:
1. Produce Name
2. Freshness Index (0-10 scale as defined below):
   0-1.9: Unsellable - Completely spoiled, showing mold, decay, or severe damage.
   2.0-3.9: Poor - Major quality issues like bruising, wilting, or discoloration.
   4.0-5.9: Fair - Some deterioration with minor blemishes or wilting.
   6.0-7.9: Good - Mostly fresh with minimal defects, suitable for sale.
   8.0-10.0: Excellent - Perfectly fresh, vibrant, and free of defects.
3. Expected Life Span (in days).

Return the output in the following JSON format:
{
    "produce_details": [
        {"Produce": "<name>", "Freshness": "<index>", "Expected_Life_Span": "<days>"},
        ...
    ]
}"""}
        ],
    }


    text = processor.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info([message])
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")


    generated_ids = model.generate(**inputs, max_new_tokens=500)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )


    if output_text:
        output = output_text[0].strip()
        cleaned_string = output.strip().replace('```json', '').replace('```', '').strip()

        try:
            print("cleaned_string is = ")
            print(cleaned_string)
            result = json.loads(cleaned_string)
            print("result is = " + str(result))
        except json.JSONDecodeError:

            result = {
                "produce_details": [
                    {"Produce": "NA", "Freshness": "NA", "Expected_Life_Span": "NA"}
                ]
            }
        return result


    return {
        "produce_details": [
            {"Produce": "NA", "Freshness": "NA", "Expected_Life_Span": "NA"}
        ]
    }

#Function to incorporate various date formats
def parse_expiry_date(expiry_date_str: str) -> datetime:
    expiry_date_str = expiry_date_str.strip().upper()


    date_formats = [
        "%m/%Y",        # MM/YYYY
        "%d/%m/%Y",     # DD/MM/YYYY
        "%d/%b/%Y",     # DD/Mon/YYYY (e.g., 05/SEP/2023)
        "%d/%b/%y",     # DD/Mon/YY (e.g., 05/SEP/24)
        "%d/%m/%y",     # DD/MM/YY (e.g., 05/02/24)
        "%m/%d/%Y",     # MM/DD/YYYY (e.g., 12/10/2024)
        "%d/%m/%Y",     # DD/MM/YYYY (e.g., 10/12/2024)
        "%Y/%m/%d",     # YYYY/MM/DD (e.g., 2024/12/10)
        "%m-%d-%Y",     # MM-DD-YYYY (e.g., 12-10-2024)
        "%d-%m-%Y",     # DD-MM-YYYY (e.g., 10-12-2024)
        "%Y-%m-%d",     # YYYY-MM-DD (e.g., 2024-12-10)
        "%b %Y",        # Mon YYYY (e.g., Dec 2024)
        "%Y %b",        # YYYY Mon (e.g., 2024 Dec)
        "%m/%Y",        # MM/YYYY (again to catch formats like '12/2024')
        "%d/%m/%Y %H:%M:%S",  # DD/MM/YYYY HH:MM:SS (e.g., 10/12/2024 14:00:00)
        "%d-%b-%Y",     # DD-Mon-YYYY (e.g., 12-DEC-2024)
        "%d/%b/%Y %H:%M",     # DD-Mon-YYYY HH:MM (e.g., 12/DEC/2024 14:00)
        "%b-%Y",        # Mon-YYYY (e.g., SEP-2024)
        "%Y-%b",        # YYYY-Mon (e.g., 2024-SEP)
    ]


    for fmt in date_formats:
        try:

            if fmt in ["%m/%Y", "%b %Y", "%Y %b", "%b-%Y", "%Y-%b"]:
                expiry_date_obj = datetime.strptime(f"01/{expiry_date_str}", "%d/%m/%Y")
            elif fmt in ["%d/%b/%Y %H:%M:%S", "%d/%b/%Y %H:%M", "%d/%m/%Y %H:%M:%S"]:

                expiry_date_obj = datetime.strptime(expiry_date_str.split()[0], fmt.split()[0])
            else:
                expiry_date_obj = datetime.strptime(expiry_date_str, fmt)
            return expiry_date_obj
        except ValueError:
            continue


    return None

@app.get("/get-database")
async def get_database():
    conn = connect_to_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM product_details")
    rows = cursor.fetchall()
    conn.close()

    current_date = datetime.now()
    products = []

    for row in rows:
        product_name = row[1]
        expiry_date = row[4]
        quantity = row[5]
        mrp = row[2]
        net_content = row[3]
        timestamp = row[6]
        expired_status = row[7]
        expected_life = row[8]

        products.append({
            "SlNo": len(products) + 1,
            "Product": product_name,
            "Timestamp": timestamp,
            "NetContent": net_content,
            "MRP": mrp,
            "ExpiryDate": expiry_date,
            "Quantity": quantity,
            "Expired": expired_status,
            "ExpectedLife": expected_life
        })

    return {"products": products}

@app.get("/get-freshdetails-database")
async def get_details_database():
    conn = connect_to_freshness_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM freshness_details")
    rows = cursor.fetchall()
    conn.close()

    produces = []

    for row in rows:

        timestamp = row[1]
        produce = row[2]
        freshness = row[3]
        expected_life = row[4]

        produces.append({
            "SlNo": len(produces) + 1,
            "Timestamp": timestamp,
            "Produce": produce,
            "Freshness": freshness,
            "Expected_Life_Span": expected_life
        })

    return {"produces": produces}

@app.post("/detailsextract-details")
async def extract_details():
    image_path = os.path.join(IMAGE_FOLDER, "captured_image.jpg")

    if os.path.exists(image_path):

        products = extract_product_details_from_image(image_path)["products"]

        current_date = datetime.now(pytz.timezone("Asia/Kolkata"))
        for product in products:
            product["Timestamp"] = current_date.isoformat()


            if product["Expiry Date"] != "NA":
                expiry_date_obj = parse_expiry_date(product["Expiry Date"])

                if expiry_date_obj is not None:

                    if expiry_date_obj.tzinfo is None:
                        expiry_date_obj = pytz.timezone("Asia/Kolkata").localize(expiry_date_obj)


                    expired_status = "Yes" if expiry_date_obj < current_date else "No"


                    months_diff = (expiry_date_obj.year - current_date.year) * 12 + expiry_date_obj.month - current_date.month
                    expected_life = f"{months_diff} month(s)" if months_diff > 0 else "NA"
                else:
                    expired_status, expected_life = "Invalid Date", "NA"
            else:
                expired_status, expected_life = "NA", "NA"


            product["Expired"] = expired_status
            product["ExpectedLife"] = expected_life


        conn = connect_to_db()
        create_table(conn)
        save_multiple_to_database(conn, products)
        conn.close()
        print(products)
        return {"products": products}

    raise HTTPException(status_code=404, detail="Image not found")

@app.post("/extract-details")
async def freshnessextract_details():
    image_path = os.path.join(IMAGE_FOLDER, "captured_image.jpg")

    if os.path.exists(image_path):

        produce_details = extract_freshness_details_for_multiple(image_path)

        current_date = datetime.now(pytz.timezone("Asia/Kolkata"))

        for produce in produce_details["produce_details"]:
            produce["Timestamp"] = current_date.isoformat()

        conn = connect_to_freshness_db()
        create_freshness_table(conn)
        save_multiple_to_freshness_database(conn, produce_details["produce_details"])
        conn.close()

        return ({"produce_details":  produce_details["produce_details"]})

    raise HTTPException(status_code=404, detail="Image not found")

@app.get("/", response_class=HTMLResponse)
async def index(request: Request):
    with open("templates/details.html", "r") as f:
        return HTMLResponse(content=f.read())

@app.get("/freshnessIndexPage", response_class=HTMLResponse)
async def freshness_index_page(request: Request):
    with open("templates/freshness.html", "r") as f:
        return HTMLResponse(content=f.read())


@app.post("/upload")
async def upload_image(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image_path = os.path.join(IMAGE_FOLDER, "captured_image.jpg")

        with open(image_path, "wb") as f:
            f.write(contents)

        return {"filename": file.filename, "status": "uploaded"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/detailsdownload-csv")
async def download_csv():
    conn = connect_to_db()
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT product_name, net_content, mrp, expiry_date, quantity, timestamp, expired, expected_life  FROM product_details")
        rows = cursor.fetchall()
        conn.close()


        if not rows:
            raise HTTPException(status_code=404, detail="No product details found in the database.")


        data = {
            "Product Name": [row[0] for row in rows],
            "Net Content": [row[1] for row in rows],
            "MRP": [row[2] for row in rows],
            "Expiry Date": [row[3] for row in rows],
            "Quantity": [row[4] for row in rows],
            "Timestamp": [row[5] for row in rows],
            "Expired": [row[6] for row in rows],
            "Expected Life": [row[7] for row in rows],

        }
        df = pd.DataFrame(data)


        stream = io.StringIO()
        df.to_csv(stream, index=False)
        stream.seek(0)


        return StreamingResponse(
            iter([stream.getvalue()]),
            media_type="text/csv",
            headers={"Content-Disposition": "attachment; filename=product_details.csv"}
        )
    except Exception as e:
        conn.close()
        raise HTTPException(status_code=500, detail=f"Error generating CSV: {str(e)}")


@app.get("/freshnessdownload-csv")
async def freshness_download_csv():
    conn = connect_to_freshness_db()
    cursor = conn.cursor()

    try:

        cursor.execute("SELECT Timestamp, Produce, Freshness, Expected_Life_Span  FROM freshness_details")
        rows = cursor.fetchall()
        conn.close()


        if not rows:
            raise HTTPException(status_code=404, detail="No freshness details found in the database.")


        data = {
            "Timestamp": [row[0] for row in rows],
            "Produce": [row[1] for row in rows],
            "Freshness": [row[2] for row in rows],
            "Expected_Life_Span": [row[3] for row in rows],

        }
        df = pd.DataFrame(data)


        stream = io.StringIO()
        df.to_csv(stream, index=False)
        stream.seek(0)  # Reset the stream pointer


        return StreamingResponse(
            iter([stream.getvalue()]),
            media_type="text/csv",
            headers={"Content-Disposition": "attachment; filename=freshness_details.csv"}
        )
    except Exception as e:
        conn.close()
        raise HTTPException(status_code=500, detail=f"Error generating CSV: {str(e)}")


# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel URL: {public_url}")

nest_asyncio.apply()
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=5000)


INFO:     Started server process [513]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


 * ngrok tunnel URL: NgrokTunnel: "https://e0e1-34-132-22-219.ngrok-free.app" -> "http://localhost:5000"
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "GET / HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 165, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-pack

INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "GET / HTTP/1.1" 200 OK
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "POST /upload HTTP/1.1" 200 OK
[{'Product Name': 'Amul BUTTER', 'MRP': '60', 'Expiry Date': '25/SEP/25', 'Net Content': 'NA', 'Quantity': 1, 'Timestamp': '2024-12-11T23:10:20.127169+05:30', 'Expired': 'No', 'ExpectedLife': '9 month(s)'}]
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "POST /detailsextract-details HTTP/1.1" 200 OK
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "POST /upload HTTP/1.1" 200 OK
[{'Product Name': 'Neem Tulsi Face Wash', 'MRP': '30.00', 'Expiry Date': 'NA', 'Net Content': '75 ml', 'Quantity': 1, 'Timestamp': '2024-12-11T23:10:33.369026+05:30', 'Expired': 'NA', 'ExpectedLife': 'NA'}]
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b05:0 - "POST /detailsextract-details HTTP/1.1" 200 OK
INFO:     2401:4900:1c5d:3f69:d8b2:3a6:3e99:9b

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [513]


In [ ]:
#Load this model only if the above model exceeds your gpu limit

# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Anamta98/Qwen2-VL-2B-Instruct-LoRA-FT")



# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Anamta98/Qwen2-VL-2B-Instruct-LoRA-FT",
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )



# min_pixels = 256*28*28
# max_pixels = 2048 * 28 * 28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)